In [ ]:
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!pip -q install seqeval
!pip -q install transformers

     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 1.3MB 14.8MB/s 
     |████████████████████████████████| 2.9MB 49.8MB/s 
     |████████████████████████████████| 890kB 45.1MB/s 
     |████████████████████████████████| 1.1MB 43.3MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm

In [ ]:
import numpy as np
import pandas as pd
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from torch.nn import CrossEntropyLoss
# dev = xm.xla_device()

if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
print(dev)
dev = torch.device(dev) 

cuda:0


In [174]:
## Possible Tags
tag_to_idx = {'O': 0, 'T-NEG': 1, 'T-POS': 2, 'T-NEU': 3, 'PAD':4}

## Data files
train_path = '/content/drive/MyDrive/sem5/NLP/train.txt'
test_path = '/content/drive/MyDrive/sem5/NLP/test.txt'
dev_path = '/content/drive/MyDrive/sem5/NLP/data/rest14/dev.txt'

idx = 0

def get_data(path):
  f = open(path, 'r')
  data = []
  labels = []
  m1 = -1
  for line in f:
    line = line.split('####')
    data.append(line[0])
    tags = []
    line[1] = line[1].split()
    m1 = max(m1,len(line[1]))
    for word in line[1]:
      word = word.split('=')
      tags.append(tag_to_idx[word[1]])
    labels.append(tags)
  return data, labels, m1

train_data, train_labels, m1 = get_data(train_path)
test_data, test_labels, m2 = get_data(test_path)
dev_data, dev_labels, m3 = get_data(dev_path)

In [ ]:
MAX_LEN = 100
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 5
lr = 1e-5
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
class dataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        sentence = str(self.sentences[index])
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        label = self.labels[index]
        label.extend([4]*200)
        label=label[:MAX_LEN]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'tags': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [175]:
train_sentences = train_data
test_sentences = test_data
training_set = dataset(tokenizer, train_sentences, train_labels, MAX_LEN)
testing_set = dataset(tokenizer, test_sentences, test_labels, MAX_LEN)
dev_set = dataset(tokenizer, dev_data, dev_labels, MAX_LEN)

In [177]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

dev_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }


training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
dev_loader = DataLoader(dev_set, **dev_params)

In [ ]:
class BERT(torch.nn.Module):
    def __init__(self, num_labels):
        super(BERT, self).__init__()
        self.num_labels = num_labels
        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', add_pooling_layer=False)
        self.dropout = torch.nn.Dropout(0.1)
        self.fc1 = torch.nn.Linear(768, 120)
        self.classifier = torch.nn.Linear(120, self.num_labels)

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        sequence_output = self.fc1(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [ ]:
model = BERT(5)
model.to(dev)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [ ]:
optimizer = transformers.AdamW(params =  model.parameters(), lr=lr)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(dev, dtype = torch.long)
        mask = data['mask'].to(dev, dtype = torch.long)
        targets = data['tags'].to(dev, dtype = torch.long)
        loss = model(ids, mask, labels = targets)[0]
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step() 

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  0.14833588898181915
Epoch: 1, Loss:  0.17099541425704956
Epoch: 2, Loss:  0.1218394860625267
Epoch: 3, Loss:  0.10677831619977951
Epoch: 4, Loss:  0.10593418776988983


In [ ]:
from seqeval.metrics import f1_score

def cal_accuracy(preds, labels):
    flat_preds = np.argmax(preds, axis=2).flatten()
    flat_labels = labels.flatten()
    return np.sum(flat_preds == flat_labels)/len(flat_labels)

In [ ]:
tag_vals = ['O', 'T-NEG', 'T-POS', 'T-NEU', 'PAD']

def valid(model, testing_loader):
    model.eval()
    eval_loss = 0; eval_accuracy = 0
    n_correct = 0; n_wrong = 0; total = 0
    predictions , true_labels = [], []
    nb_eval_steps, nb_eval_examples = 0, 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(dev, dtype = torch.long)
            mask = data['mask'].to(dev, dtype = torch.long)
            targets = data['tags'].to(dev, dtype = torch.long)

            output = model(ids, mask, labels=targets)
            loss, logits = output[:2]
            logits = logits.detach().cpu().numpy()
            label_ids = targets.to('cpu').numpy()
            predictions.append([list(p) for p in np.argmax(logits, axis=2)])
            true_labels.append(label_ids)
            accuracy = cal_accuracy(logits, label_ids)
            eval_loss += loss.mean().item()
            eval_accuracy += accuracy
            nb_eval_examples += ids.size(0)
            nb_eval_steps += 1
        eval_loss = eval_loss/nb_eval_steps
        return predictions, true_labels

In [178]:
# To get the results on the validation set. This data is not seen by the model
p,t = valid(model, dev_loader)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [179]:
y_true = []
y_pred = []
for batch in range(len(t)):
  for entry in range(len(t[batch])):
    N = 0
    while t[batch][entry][N] != 4 and N<len(p[batch][entry]) and N<len(t[batch][entry]):
      N+=1 
    try:
      a = [tag_vals[p[batch][entry][i]] for i in range(N)]
      y_pred.append(a)
      b = [tag_vals[t[batch][entry][i]] for i in range(N)]
      y_true.append(b)
    except:
      print("Error")

In [180]:
print("F1-Score: {}".format(f1_score(y_pred, y_true)))

F1-Score: 0.22784810126582278


/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: T-POS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: T-NEU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: T-NEG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [181]:
num = 0
den = 0
for i in range(len(y_true)):
  for j in range(len(y_true[i])):
    if y_true[i][j] == y_pred[i][j]:
      num += 1
    den +=1
print('Accuracy:', (num/den)*100)

Accuracy: 86.86393185384443


In [ ]:
from seqeval.metrics import classification_report

In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          AD       0.00      0.00      0.00         0
         NEG       0.19      0.22      0.21       196
         NEU       0.21      0.11      0.14       198
         POS       0.38      0.29      0.33       728

   micro avg       0.27      0.24      0.25      1122
   macro avg       0.19      0.15      0.17      1122
weighted avg       0.31      0.24      0.27      1122



/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: T-POS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: T-NEU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: T-NEG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, ms

In [ ]:
PATH = '/content/drive/MyDrive/sem5/NLP/absa_v3.sav'
## Save
torch.save(model.state_dict(), PATH)
## Load
# model = BERT()
# model.load_state_dict(torch.load(PATH))
# model.eval()

TEST WITH YOUR SENTENCES

In [212]:
sentence = 'i did not like the muffins here'
inputs = tokenizer(sentence, return_tensors="pt")
id = inputs['input_ids'].to(dev)
att = inputs['attention_mask'].to(dev)
output = model(id, att)
logits = output[0]
logits = logits.detach().cpu().numpy()
a = [list(p) for p in np.argmax(logits, axis=2)]
out = a[0]
out = [tag_vals[i] for i in out]

asp = []
inpu = sentence.split()
for i in range(min(len(out), len(inpu))):
  if out[i] != 'O' and out[i] !='PAD':
    asp.append((inpu[i],out[i][2:]))

In [213]:
sent = {'NEG':'Negative', 'POS': 'Positive', 'NEU':'Neutral'}

In [214]:
for entry in asp:
  print('Aspect: {}, Sentiment: {}'.format(entry[0], sent[entry[1]]))

Aspect: muffins, Sentiment: Negative
